<a href="https://colab.research.google.com/github/ArazShilabin/Poem-Generation-on-Shakespeare/blob/master/poem_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x # GOOGLE COLLAB USERS ONLY
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # GOOGLE COLLAB USERS ONLY`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


**get data**

In [4]:
dir_text = os.getcwd()+'/shakespeare.txt'
text = open(dir_text, 'r').read()
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

**getting and mapping our unique characters**

In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)
print(vocab[0])

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']




## PreProcessing

In [6]:
char_to_ind = {word:ind for ind, word in enumerate(vocab)}
ind_to_char = np.array(vocab)
print(char_to_ind['R'])
print(ind_to_char[43])

43
R


In [7]:
encoded_text = np.array([char_to_ind[ch] for ch in text])
print(encoded_text.shape)
print(encoded_text[:100])


(5445609,)
[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64]


**choosing batch size**

In [8]:
print(text[:200])
print(len(text))


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
 
5445609


In [9]:
# we need to see at least 3 lines to see the poem rhyme
good_sample = "\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,"
len(good_sample)

157

In [10]:
# I chose 120 as a median
seq_len = 120
total_num_seq = len(text)//(seq_len+1)
total_num_seq

45005

In [11]:
# Create Training Sequences (we mean turn 5445609 to tensor)
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
for i in char_dataset.take(50): # it's the same data but in a tensor format printing first 50 chars, that's why i.numpy() is used
     print(ind_to_char[i.numpy()])




 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w


In [12]:
# the 45005 we have isn't exactly 45005, we rounded it down, so we have to get rid of the remaining reminder so we chose True
sequences = char_dataset.batch(seq_len+1, drop_remainder=True) # len+1 because index starts from [0 to seq_len]
sequences

<BatchDataset shapes: (121,), types: tf.int64>

In [0]:
def create_seq_targets(seq): # 12345
    input_txt = seq[:-1] # 1234
    target_txt = seq[1:] # 2345
    return input_txt, target_txt

dataset = sequences.map(create_seq_targets)


In [14]:

for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()])) # str.join(list): it joins all the elements in list, putting str between them
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))


[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [15]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle the entire sequence in memory.
# Instead, it maintains a buffer in which it shuffles elements.
# if this doesn't exist, in high memory data it would cause an error (overflow)
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset # ((128, 120), (128, 120)) because each of the tuple are for input text and output text, the 128 is for batch size and 120 for length of the sentence


<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab) # 84

# The embedding dimension
embed_dim = 64

# Number of RNN units (RNN size)
rnn_neurons = 1026

In [0]:
# we use sparse because we are one-hot encoded
from tensorflow.keras.losses import sparse_categorical_crossentropy 

def sparse_cat_loss(y_true,y_pred): # we made this func so we can make from_logits=True
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True) # logits is the logarithmic probabilites

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [0]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform')) # glorot is the last name of xavier, it's the same initilization
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [0]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


**checking if the model works correct**

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    # Predict off some random batch
    example_batch_predictions = model(input_example_batch)

    # Display the dimensions of the predictions
    print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")
    print("\nthis is the predicted probabilities on the first batch on the 6th vocab which are semi random because it hasn't been trained yet:\n",example_batch_predictions[0][5])
    

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)

this is the predicted probabilities on the first batch on the 6th vocab which are semi random because it hasn't been trained yet:
 tf.Tensor(
[-4.77511296e-03  6.38767669e-04 -4.49275598e-03 -1.16717387e-02
 -7.01847719e-03 -8.48157052e-03  3.20284721e-03  4.05522203e-03
 -5.88445272e-03  4.33088047e-03  3.44032864e-03 -2.49285786e-03
  2.30217283e-03 -6.41316548e-03 -3.64438375e-03 -1.83474831e-03
  7.12830108e-03 -2.88953865e-03  4.47027525e-03 -6.66826684e-03
  5.15616313e-03  3.08726425e-03  4.43988387e-03 -1.03964964e-02
 -4.47107101e-04 -1.62357220e-03 -8.35289899e-03 -8.74937885e-03
 -7.12859794e-04  2.36988859e-03  5.14071248e-03 -4.99784062e-03
 -9.30446945e-03  5.92658995e-03  8.90134461e-03 -2.92440900e-03
 -7.14827096e-03  4.43589641e-03  2.94708507e-03 -1.13544408e-02
  2.69004260e-03 -1.68926350e-03 -5.74308028e-03  2.35468315e-06
 -3.33795906e-03  7.63794826e-03  3.42725980e-04 -3.02504911e-03
  9.19934129e-

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # tf.random.categorical draws a sample on random on a probability distribution
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices


array([ 6, 45, 79, 54, 54, 73, 26, 25, 51, 14, 29,  0, 36, 61, 58, 42, 31,
       42, 52, 23, 12, 57, 60, 31, 77, 81, 61, 66, 16, 67,  6, 10, 59, 10,
       29, 38, 45, 81,  0,  8, 39, 45, 65, 71, 16,  3, 27, 63, 42,  0, 51,
       26, 33, 62, 70, 34, 71, 71, 32, 21,  5, 24, 52, 70, 24, 77,  6, 59,
       57, 30, 66, 30, 13, 54, 26, 26, 71, 20, 38, 17, 76, 14, 18, 46, 32,
       34, 44,  7, 61, 27, 33, 20,  7, 15, 33, 60, 13, 59, 38, 41, 79, 27,
       25, 61, 43,  3, 23,  5, 29, 35, 41, 11, 62, 61, 51, 18, 79, 51,  7,
       37])

In [24]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

ll me how thou hast dispos'd thy charge.
DROMIO OF EPHESUS. My charge was but to fetch you from the mart
  Home to your 


Next Char Predictions: 

(Tx__rA?Z3D
KfcQFQ[<1beFvzfk5l(.d.DMTz
,NTjp5"BhQ
ZAHgoIppG:'>[o>v(dbEkE2_AAp9M6u37UGIS)fBH9)4He2dMPxB?fR"<'DJP0gfZ7xZ)L


## Train

In [25]:
epochs = 30
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 88s 252ms/step - loss: 2.5441
Epoch 2/30
351/351 [==============================] - 88s 251ms/step - loss: 1.7187
Epoch 3/30
351/351 [==============================] - 88s 252ms/step - loss: 1.4496
Epoch 4/30
351/351 [==============================] - 88s 252ms/step - loss: 1.3344
Epoch 5/30
351/351 [==============================] - 89s 253ms/step - loss: 1.2733
Epoch 6/30
351/351 [==============================] - 88s 252ms/step - loss: 1.2335
Epoch 7/30
351/351 [==============================] - 89s 253ms/step - loss: 1.2039
Epoch 8/30
351/351 [==============================] - 88s 252ms/step - loss: 1.1802
Epoch 9/30
351/351 [==============================] - 89s 252ms/step - loss: 1.1602
Epoch 10/30
351/351 [==============================] - 89s 253ms/step - loss: 1.1429
Epoch 11/30
351/351 [==============================] - 89s 253ms/step - loss: 1.1264
Epoch 12/30
351/351 [==============================] - 89s 252ms/step - lo

**load-save the model**

In [0]:
# saving the model
model.save('shakespeare_gen.h5')

In [0]:
# loading the saved model
from tensorflow.keras.models import load_model

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1) # we are basically changing the previous model to something with batch_size of 1 because we are going to test it, not train it anymore
model.load_weights('shakespeare_gen.h5')
model.build(tf.TensorShape([1, None])) # in build we are specifying the input tensor shape


##Generating Poem (output)

In [0]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''

    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Lower temperature == less surprising/ more expected
    # Higher temperature == more surprising / less expected
    
    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):

        # Generate Predictions
        predictions = model(input_eval)

        # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Transform back to character letter
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))


In [29]:
# example 1: tmp=1.0
print(generate_text(model,"flower",gen_size=1000))

flower.
    Who's here! Here's the stage
    Of one, Scome, spirit, friend, at least, thou diest.

              blood her love.
  TRANIO. Grant him in thine own boy's hearing.
  SHALLOW. He is dead, most proof. My letter hath ick to  
    thoughts may be so reasonable- and there is no jottlu under Taunt sir, a  
    fountain point.
  CALIBAN. Speak to Antonio; an increase, boy, but I'll fithey
    So marial Toby trimul and full of great
    Must cuck. Fire and sorrow at Ethise will bear
    Fire and the Tribune -and these men should say anythough about your fillogs.
  MALVOLIO. 'Remember her to move; therefore my protection.              Exeunt SERVANTS

  MESSENGER. Dost thou hear me, de end?
    Earth, good Sister, and myself and yours,
    And theess yet let it be most fierce.
  HEEMIO. As much as I become enough
    so such a man good innocent eyes? who supper to conspire against conduct
    Divided into the court still be the world,
    And that they were inform'd steel for perfu

In [32]:
# example 1: temp=0.4, less suprising
print(generate_text(model,"flower",gen_size=1000,temp=0.4))

flower.
    The bastard discourse of his deserts in this letter to my will.
  ANTONIO. The day should be the faults and the conclusion
    Of all these books. What though I know 'tis now the devil?
  POLIXENES. Sir, you are near to her; now be the poor maid, by me
    I stood before the best of men are but
    As I am liberal in the court of Jerus' house,
    And they know the rest of the thing to meet him.
    For the entreaty and whose great ones lost
    The storm of royal rooth and heir of ingrateful service
    Who should have been a mortal to deliver
    A better wishes that have done with me.
    Now is the business is to be reveng'd.
    I am a gentleman of your deigning.
                                                             Exeunt all but BUCKINGHAM and CAMILLO
  SERVANT. What is the matter?
  PANDARUS. What a circumstance good men! What is your will?
  TIMON. The boy hath been best known with my desires,
    And he shall say I have no power to do it.
    I have seen th